In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from scipy import stats
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import math

from xgboost import plot_importance, plot_tree

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

# select the style from fivethityeight website
plt.style.use('fivethirtyeight') 
mpl.rcParams['lines.linewidth'] = 2
# mpl.rcParams['axes.labelsize'] = 14
# mpl.rcParams['xtick.labelsize'] = 12
# mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['text.color'] = 'k'

# predefined figsize
figsize=(12,9)

np.random.seed(238746)

plt.style.use('seaborn-dark-palette')
mypal = plt.rcParams['axes.prop_cycle'].by_key()['color'] # Grab the color pal



#1608 da escludere always

# settings
import warnings
warnings.filterwarnings("ignore")


In [ ]:
train_df = pd.read_csv('../train.csv', sep=";")
train_df = train_df.rename({"Unnamed: 0":"date"}, axis=1).dropna()
train_df = train_df.rename({"volume_on_promo w-1":"promo"}, axis=1).dropna()
train_df["date"] = train_df["date"].apply(lambda x:datetime.strptime(x[3:], '%d %B %Y'))
train_df.set_index('date')

df2 = pd.DataFrame(columns= train_df.columns)

for sku in train_df['sku'].unique():
    sales_sku = train_df.loc[train_df['sku'] == sku]
    sales_sku['diff1'] = sales_sku['sales w-1'] - sales_sku['sales w-1'].shift(1)
    df2 = pd.concat((df2, sales_sku), axis = 0)
    
df2['quarter'] = df2['date'].dt.quarter
df2['month'] = df2['date'].dt.month
df2['year'] = df2['date'].dt.year
df2['dayofyear'] = df2['date'].dt.dayofyear
df2['dayofmonth'] = df2['date'].dt.day
df2['weekofyear'] = df2['date'].dt.weekofyear

#train_df = df2
train_df = df2.dropna()

train_df.set_index('date', inplace=True)

train_df

In [ ]:
test_df = pd.read_csv('../x_test.csv')
test_df = test_df.rename({"Unnamed: 0":"date"}, axis=1).dropna()
test_df = test_df.rename({"volume_on_promo w-1":"promo"}, axis=1).dropna()
test_df["date"] = test_df["date"].apply(lambda x:datetime.strptime(x[3:], '%d %B %Y'))
test_df.set_index('date')
test_df

In [ ]:

sku_unique = train_df.sku.unique()
sku_train = train_df.loc[train_df["scope"]==0].sku.unique()
sku_test = train_df.loc[train_df["scope"]==1].sku.unique()

sku_unique

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

#plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

sku_id = sku_unique[12]

# Original Series
#fig, axes = plt.subplots(3, 2, sharex=True)

plt.plot(train_df.loc[train_df["sku"]==sku_id].target)
plt.title('Original Target')
plot_acf(train_df.loc[train_df["sku"]==sku_id].target)
plt.title('Original Autocorrelation')
plot_pacf(train_df.loc[train_df["sku"]==sku_id].target)
plt.title('Original Partial Autocorrelation')

plt.show()

# 1st Differencing
plt.plot(train_df.loc[train_df["sku"]==sku_id].target.diff())
plt.title('1st Order Differencing')
plot_acf(train_df.loc[train_df["sku"]==sku_id].target.diff().dropna())
plt.title('1st Order Autocorrelation')
plot_pacf(train_df.loc[train_df["sku"]==sku_id].target.diff().dropna())
plt.title('1st Order Partial Autocorrelation')

plt.show()

# 2nd Differencing
plt.plot(train_df.loc[train_df["sku"]==sku_id].target.diff().diff())
plt.title('2nd Order Differencing')
plot_acf(train_df.loc[train_df["sku"]==sku_id].target.diff().diff().dropna())
plt.title('2nd Order Autocorrelation')
plot_pacf(train_df.loc[train_df["sku"]==sku_id].target.diff().diff().dropna())
plt.title('2nd Order Partial Autocorrelation')

plt.show()



In [ ]:
# SPLITTO SERIE IN TRAIN E VAL
sku_id = 2698
#sku_id = sku_unique[9]
train_df.loc[train_df["sku"]==sku_id].target[0:105].plot(label="train")
train_df.loc[train_df["sku"]==sku_id].target[104:].plot(label="val")
plt.legend()

df_sku_train = train_df.loc[train_df["sku"]==sku_id].target[0:101]
df_sku_val = train_df.loc[train_df["sku"]==sku_id].target[100:]

In [ ]:
# Build Model
from statsmodels.tsa.arima_model import ARIMA
#model = ARIMA(df_sku_train, order=(1,1,1)) 
model = ARIMA(df_sku_train, order=(3,1,2))
#model = ARIMA(df_sku_train, order=(1, 1, 1))  
fitted = model.fit(disp=-1)  
print(fitted.summary())

# Forecast
fc, se, conf = fitted.forecast(31, alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=df_sku_val.index)
lower_series = pd.Series(conf[:, 0], index=df_sku_val.index)
upper_series = pd.Series(conf[:, 1], index=df_sku_val.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(df_sku_train, label='training')
plt.plot(df_sku_val, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
# Visualizzo il target di ogni sku
for sku_id in sku_unique:
    train_df.loc[train_df["sku"]==sku_id].target.plot(figsize=(8,6))
    plt.title("sku {}".format(sku_id))
    plt.show()

In [ ]:
sku_autocorr_list = []
for sku_id in sku_unique:
    correl_target = train_df.loc[train_df["sku"]==sku_id]["target"].autocorr()
    correl_diff1 = train_df.loc[train_df["sku"]==sku_id]["diff1"].autocorr()
    sku_autocorr_list.append((sku_id, "%.2f" % correl_target, "%.2f" % correl_diff1))
print(sku_autocorr_list)

In [ ]:
for sku_id in sku_train:
    plt.plot(train_df.loc[train_df["sku"]==sku_id]["target"], label="target");
    plt.plot(train_df.loc[train_df["sku"]==sku_id]["diff1"], label="diff1");
    plt.legend()
    plt.title(sku_id)
    plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller

for sku_id in sku_unique:
    adf_t = adfuller(train_df.loc[train_df["sku"]==sku_id]["target"])
    adf_d = adfuller(train_df.loc[train_df["sku"]==sku_id]["diff1"])
    print("{}".format(sku_id))
    print("p-value of : target: {0:.5f}".format( adf_t[1]))
    print("p-value of : diff1 {0:.5f}".format(adf_d[1]))

# target è random walk, mentre diff1 no, è un white noise?

In [ ]:
#ARMA MODEL
# simulates an AR(1) model, the zero refers to the MA model (see next)
from statsmodels.tsa.arima_model import ARMA
#ARMA PROCESS
from statsmodels.tsa.arima_process import ArmaProcess
np.random.seed(0)